In [34]:
#@title Install Packages
!pip install SimpleITK
!pip install skan
!pip install tifffile

In [32]:
#@title Copy Data from Google Drive
!cp -r /content/drive/MyDrive/mydata/sea_urchin_data/3D/Segmentation /content/
!cp -r "/content/drive/MyDrive/mydata/sea_urchin_data/3D/Modello 1 Galleria Piastra.obj" /content/

In [49]:
#@title Import Packages
import tifffile as tiff
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import SimpleITK as sitk
import glob
import plotly.express as px

In [4]:
#@title Define load_img_from_tiff(path2img)
def load_img_from_tiff(path2img):
    """
    Parameters
    ----------
    path2img: str 
        path to image Tiff file
    
    Returns
    -------
    img_array: np.array
        image data in numpy format
    """
    img = sitk.ReadImage(path2img)
    img_array = sitk.GetArrayFromImage(img)
    return(img_array)

In [7]:
path = "/content/drive/MyDrive/mydata/sea_urchin_data/3D/Galleria Piastra Modello 1 100x100 Echi1-10x.tif"
data = load_img_from_tiff(path)
for i in range(len(data)):
    data[i] = data[i]*(255.0/data[i].max())

In [27]:
blurred_data = np.zeros_like(data)
for i in range(len(blurred_data)):
    blurred_data[i] = cv2.GaussianBlur(data[i], (3,3),3)

thresholded_data = np.zeros_like(data)
for i in range(len(thresholded_data)):
    bin, thresholded_data[i] = cv2.threshold(blurred_data[i],175, 255, cv2.THRESH_BINARY)

In [28]:
filelist = glob.glob('Segmentation/*.tiff') 
filename_prefix = filelist[0][:-8]
filename_suffix = filelist[0][-5:]
number_ids=[]

for i in range(len(filelist)):
    number_ids.append(int(filelist[i][-8:-5]))
    filelist[i] = filelist[i][-8:-5]

filelist.sort()
sorted_filelist=[]

for j in range(len(filelist)):
    sorted_filelist.append(filename_prefix+filelist[j]+filename_suffix)

val_seg = np.array([plt.imread(fname) for fname in sorted_filelist])

In [ ]:
seg_images = np.array([val_seg*255, thresholded_data, data])
fig = px.imshow(seg_images, 
                facet_col=0,
                animation_frame=1,
                color_continuous_scale ="gray")

for i, label in enumerate(['Seg3D', 'OpenCV (Python)', "Raw"]):
    fig.layout.annotations[i]['text'] = label

# fig.layout.template = 'plotly_dark'
fig.show()

In [ ]:
tiff.imsave("threshold.tiff", thresholded_data)
test_data = load_img_from_tiff("threshold.tiff")
plt.imshow(test_data[0])